In [1]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot


import requests

from tqdm import tqdm_notebook as tqdm

## Convert from PDF

In [2]:
dd = read_pdf('../files/Prestadores de Serviço - Agosto 2020.pdf', pages='all' ,multiple_tables=True, encoding='utf-8', lattice=False)

In [3]:
df = pd.DataFrame()

for i in range(len(dd)):
    ddf = dd[i]
    first_row = pd.DataFrame(ddf.columns.tolist()).T
    first_row.columns = [i for i in range(len(first_row.columns))]
    ddf.columns = [i for i in range(len(ddf.columns))]
    ddf = pd.concat([first_row,ddf],0)
    
    df = pd.concat([df,ddf], axis=0)
df = df.reset_index(drop=True)


In [4]:
df.columns = df.iloc[0].str.replace(" ", "_").str.lower().tolist()

mask = df['nome']!='NOME'
df = df[mask]

df['valor'] = df['valor'].str.replace('.','').str.replace(',','.').astype(float)
df['cnpj_number'] = df['cnpj'].str.replace('.','').str.replace('/','').str.replace('-','')

## Get Socios From BrIO

In [7]:
# socios = pd.read_csv('../../1databases/socio.csv')


In [13]:
mask = socios['cnpj'].isin(df['cnpj_number'].unique().tolist())
df_socios = socios[mask]

In [19]:
df_socios.to_csv('../data/cnpj_socios_santa_casa_pacaembu/socios_prestadoras_santa_casa_pacaembu.csv', index=False)

In [34]:
total = len(df['cnpj_number'].unique().tolist())
part = len(df_socios['cnpj'].unique())

print(part)
print(total)

294
348


In [31]:
df['cnpj_encontrado'] = np.where(df['cnpj_number'].astype(int).isin(df_socios['cnpj'].unique().tolist()), 'sim','nao')

In [33]:
df.to_csv('../data/cnpj_socios_santa_casa_pacaembu/prestadoras_santa_casa_pacaembu.csv', index=False)

In [21]:
df_final = pd.DataFrame()
ne = df['cnpj_number'].unique().tolist()
cnpjs_not_find = []

for cnpj in ne:
    url = f'https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj={cnpj}'

    r = requests.get(url)

    if r.status_code != 200:
        print(len(ne),' - ',f'{cnpj} nao encontrado')
        cnpjs_not_find.append(cnpj)
    else:
#             print(len(ne),' - ',url)
        j = r.json()
        dd = pd.DataFrame.from_dict(j['results'])


        df_final = pd.concat([df_final, dd], axis=0)

        ne.remove(cnpj)

# df_final.to_csv('test.csv',index=False, mode='a')

338  -  00040925396818 nao encontrado
338  -  24377858000138 nao encontrado
338  -  34036737000186 nao encontrado
338  -  00002342733569 nao encontrado
338  -  00937785000123 nao encontrado
338  -  20526353000137 nao encontrado
338  -  34389503000112 nao encontrado
338  -  11839184000102 nao encontrado
338  -  11364111000101 nao encontrado
338  -  50236157000199 nao encontrado
338  -  12869838000286 nao encontrado
338  -  04543018000190 nao encontrado
338  -  00039627684830 nao encontrado
338  -  31765685000190 nao encontrado
338  -  48791685000168 nao encontrado
338  -  27510103000158 nao encontrado
338  -  61600839000155 nao encontrado
338  -  06278001000189 nao encontrado
338  -  00031731869827 nao encontrado
338  -  28954520000152 nao encontrado
338  -  00941152000199 nao encontrado
338  -  61363032000146 nao encontrado
338  -  00000343143186 nao encontrado
338  -  04228124000180 nao encontrado
338  -  47193115000103 nao encontrado
338  -  61418042000131 nao encontrado
338  -  0926

KeyboardInterrupt: 